# Predicting Progression of Alzheimer's Disease Autoencoder (PPAD-AE)

In [1]:
!pip install keras-tcn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.0 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [2]:
# Import necessary libraries
import numpy as np
from pandas import read_csv
import pandas as pd
import random
from keras.models import Sequential, Model
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score,accuracy_score,f1_score,roc_auc_score
from keras.regularizers import l1, l2
from keras.layers import Bidirectional
from keras.layers import Dense, SimpleRNN, concatenate, Input, Flatten
from keras.layers import GRU
from keras.layers import Dropout 
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Masking
from tcn import tcn
from sklearn.metrics import f1_score
from tcn import tcn

import tensorflow as tf
import keras
import pickle

# A customized binary cross entropy loss function
#𝐿𝑜𝑠𝑠 = −1/𝑁 ∑(𝛼 ∙ (𝑦 ∙ 𝑙𝑜𝑔 𝑦′)) + ((1 − 𝛼) ∙ (1 − 𝑦) ∙ 𝑙𝑜𝑔(1 − 𝑦′))

In [3]:
# to panelize positive (converter) misclassification
def binary_cross_entropy(y, yhat):
    alpha = 0.7
    loss = -(tf.math.reduce_mean((alpha * y * tf.math.log(yhat + 1e-6)) + ((1.0- alpha) * (1 - y) * tf.math.log(1 - yhat + 1e-6)), axis=-1))
    return loss

# PPAD-AE

In [4]:
# PPAD method that takes the follwing parametres:
# cell: it represents the RNN cell will be used {'GRU', 'biGRU', 'LSTM', 'biLSTM'}
# drout: it represents the drop out rate will be used {0, 0.1, 0.2, 0.3, 0.4, 0.5}
# L2: it represents the L2 regularization {0.1, 0.001, 0.00001, 0.0000001}
# ftp: it represents future time point to predict in PPAD-AE its {2, 3, 4}
def PPAD_AE_with_demographic(cell, drout, L2, ftp):
    batch_shape = (None, time_steps, num_features_in_each_time_step)
    model = Sequential()
    model.add(Masking(-1, batch_input_shape=batch_shape))
    
    if cell == 'biGRU':
        model.add(Bidirectional(GRU(16, activity_regularizer=l2(L2), return_sequences=True, activation='relu')))
        model.add(Dropout(drout))
        model.add(Bidirectional(GRU(8, activity_regularizer=l2(L2), return_sequences=False, activation='relu')))
        model.add(Dropout(drout))
        model.add(RepeatVector(ftp))
        model.add(Bidirectional(GRU(8, activity_regularizer=l2(L2), return_sequences=True, activation='relu')))#activation = 'tanh'
        model.add(Dropout(drout))
        model.add(Bidirectional(GRU(16, activity_regularizer=l2(L2), return_sequences=False, activation='relu')))
        model.add(Flatten())
        model.add(Dropout(drout, name='out'))
    elif cell == 'biLSTM':
        model.add(Bidirectional(LSTM(16, activity_regularizer=l2(L2), return_sequences=True, activation='relu')))
        model.add(Dropout(drout))
        model.add(Bidirectional(LSTM(8, activity_regularizer=l2(L2), return_sequences=False, activation='relu')))
        model.add(Dropout(drout))
        model.add(RepeatVector(ftp))
        model.add(Bidirectional(LSTM(8, activity_regularizer=l2(L2), return_sequences=True, activation='relu')))#activation = 'tanh'
        model.add(Dropout(drout))
        model.add(Bidirectional(LSTM(16, activity_regularizer=l2(L2), return_sequences=False, activation='relu')))
        model.add(Flatten())
        model.add(Dropout(drout, name='out'))
    elif cell == 'TCN' :        
        model.add((tcn.TCN(16, activity_regularizer= l2(L2), return_sequences=True, dilations=[1, 2], activation='relu')))
        model.add(Dropout(drout))
        model.add((tcn.TCN(8, activity_regularizer= l2(L2), return_sequences=False,  dilations=[1, 2],activation='relu')))
        model.add(Dropout(drout))
        model.add(RepeatVector(ftp))
        model.add((tcn.TCN(8, activity_regularizer=l2(L2), return_sequences=True,  dilations=[1, 2],activation='relu')))#activation = 'tanh'
        model.add(Dropout(drout))
        model.add((tcn.TCN(16, activity_regularizer=l2(L2), return_sequences=False, dilations=[1, 2], activation='relu')))
        model.add(Flatten())
        model.add(Dropout(drout, name='out'))
    elif cell == 'GRU':
        model.add(GRU(16, activity_regularizer=l2(L2), return_sequences=True, activation='relu'))
        model.add(Dropout(drout))
        model.add(GRU(8, activity_regularizer=l2(L2), return_sequences=False, activation='relu'))
        model.add(Dropout(drout))
        model.add(RepeatVector(ftp))
        model.add(GRU(8, activity_regularizer=l2(L2), return_sequences=True, activation='relu'))#activation = 'tanh'
        model.add(Dropout(drout))
        model.add(GRU(16, activity_regularizer=l2(L2), return_sequences=False, activation='relu'))
        model.add(Flatten())
        model.add(Dropout(drout, name='out'))
    elif cell == 'LSTM':
        model.add(LSTM(16, activity_regularizer=l2(L2), return_sequences=True, activation='relu'))
        model.add(Dropout(drout))
        model.add(LSTM(8, activity_regularizer=l2(L2), return_sequences=False, activation='relu'))
        model.add(Dropout(drout))
        model.add(RepeatVector(ftp))
        model.add(LSTM(8, activity_regularizer=l2(L2), return_sequences=True, activation='relu'))#activation = 'tanh'
        model.add(Dropout(drout))
        model.add(LSTM(16, activity_regularizer=l2(L2), return_sequences=False, activation='relu'))
        model.add(Flatten())
        model.add(Dropout(drout, name='out'))
    
    #Demographic model
    model2 = Input(shape=(demographic_features))
    
    # concatenating RNN output with demographic data
    concat = concatenate([model.get_layer(name='out').output, model2], name='Concatenate')
    
    # MLP Classification model
    final_model_ = Dense(16, activation='relu')(concat)
    final_model0 = Dense(8, activation='relu')(final_model_)
    final_model1 = Dense(4, activation='relu')(final_model0)
    final_model2 = Dense(1, activation='sigmoid')(final_model1)
    final_model = Model(inputs=[model.inputs, model2], outputs=final_model2, name='Final_output')
    
    final_model.compile(loss= binary_cross_entropy, optimizer='adam',metrics=['accuracy'])
    return final_model

# F2 Calculation

In [5]:
# fbeata_function method to calculate f2 score
def overall_fbeta_function(pred, actual):
    # reshape the output
    if len(actual.shape) > 2:
        actual = np.reshape(actual, (actual.shape[0], actual.shape[1]*actual.shape[2]))
    
    y = []
    
    for i in range(pred.shape[0]):
        for j in range(pred.shape[1]):
            if pred[i,j] > 0.5:
                pred[i,j] = 1
            else:
                pred[i,j] = 0 
    
    for i in range(pred.shape[0]):
        COUNTER = 0
        while (COUNTER < actual.shape[1]):
            if actual[i,COUNTER] != -1:
                COUNTER+=1
            else:
                break
        y.append(actual[i,COUNTER-1])
    y = np.array(y) 
    y = np.reshape(y, (y.shape[0], 1))
    
    return fbeta_score(y, pred, beta=2)

In [46]:
# train and evaluate model
def do_PPAD_AE(longitudinal_train_data, train_label, longitudinal_test_data, test_label, demographic_train_data,
               demographic_test_data, iteration, ftp, hp_list):
    X_train = longitudinal_train_data
    y_train = train_label[:,-1]

    X_test = longitudinal_test_data
    y_test = test_label[:,-1]
    
    # hp
    batch_size_ = int(hp_list[0])
    epochs_ = int(hp_list[1])
    drout = hp_list[2]
    L2 = hp_list[3]
    
    cell = "LSTM"
    
    print(cell)
#     cell = hp_list[4].strip()
    
    model = PPAD_AE_with_demographic(cell, drout, L2, ftp)
    history = model.fit([X_train, demographic_train_data], y_train, epochs=epochs_, batch_size = batch_size_,
                        shuffle = True, verbose=0)
    
    #train
    train_loss, train_acc = model.evaluate([X_train, demographic_train_data], y_train, batch_size = batch_size_, verbose=0)
    train_pred = model.predict([X_train, demographic_train_data], verbose=0)
    print('Training is over')
    
    #test
    test_loss, test_acc = model.evaluate([X_test, demographic_test_data], y_test, batch_size = batch_size_, verbose=0)
    test_pred = model.predict([X_test, demographic_test_data], verbose=0)
    print('Test is over')
    
    # prepare results

    for i in range(test_pred.shape[0]):
        for j in range(test_pred.shape[1]):
            if test_pred[i,j] > 0.5:
                test_pred[i,j] = 1
            else:
                test_pred[i,j] = 0
                
    predicted_l = np.zeros((len(test_pred)))
    real_l = np.zeros((len(y_test)))
    dx = test_pred.shape[1] - 1
    for i in range(len(test_pred)):
        predicted_l[i] = test_pred[i,dx]
    for i in range(len(y_test)):
        real_l[i] = y_test[i,dx]
    CM = confusion_matrix(real_l, predicted_l, labels=[0,1])
    
    
    print("predicted_l")
    print(predicted_l)
    print("=====================================")
    
    print("real_l")
    print(real_l)
    print("=====================================")
    
    
    CM = confusion_matrix(real_l, predicted_l, labels=[0,1])
    
    print("CM")
    print(CM)
    print("=====================================")
    
    sensitivity = CM[1,1] / (CM[1,1] + CM[1,0])
    specificity = CM[0,0] / (CM[0,0] + CM[0,1])
    
    F1_Score = f1_score(real_l, predicted_l, average='binary')
    
    print("F1_Score")
    print(F1_Score)
    print("=====================================")

    
    # Table of results
    col = 'Iteration '+str(iteration)
    metrics_results_df = pd.DataFrame(columns = [col])

    metrics_results_df.loc[len(metrics_results_df)] = [round(accuracy_score(y_test[:, -1], test_pred[:, -1]), 3)]
    metrics_results_df.loc[len(metrics_results_df)] = [round(test_loss, 3)]
    metrics_results_df.loc[len(metrics_results_df)] = [round(roc_auc_score(y_test[:, -1], test_pred[:, -1]), 3)]
    metrics_results_df.loc[len(metrics_results_df)] = [round(fbeta_score(y_test[:, -1], test_pred[:, -1], beta=2), 3)] 
    metrics_results_df.loc[len(metrics_results_df)] = [round(sensitivity, 3)] 
    metrics_results_df.loc[len(metrics_results_df)] = [round(specificity, 3)]  
    metrics_results_df.loc[len(metrics_results_df)] = [round(train_acc, 3)]
    metrics_results_df.loc[len(metrics_results_df)] = [round(train_loss, 3)]
    metrics_results_df.loc[len(metrics_results_df)] = [round(overall_fbeta_function(train_pred, y_train), 3)]
    
#     print(metrics_results_df)
    
    return metrics_results_df

In [47]:
# Method to create and return an empty dataframe for results 
def create_table():
    # Table of results
    PPAD_metrics_results_df = pd.DataFrame(columns = ['Metrics'])
    PPAD_metrics_results_df.loc[len(PPAD_metrics_results_df)] = ['Accuracy (Test)']
    PPAD_metrics_results_df.loc[len(PPAD_metrics_results_df)] = ['Loss (Test)']
    PPAD_metrics_results_df.loc[len(PPAD_metrics_results_df)] = ['ROC_AUC (Test)']
    PPAD_metrics_results_df.loc[len(PPAD_metrics_results_df)] = ['F-2 (Test)'] 
    PPAD_metrics_results_df.loc[len(PPAD_metrics_results_df)] = ['Sensitivity (Test)'] 
    PPAD_metrics_results_df.loc[len(PPAD_metrics_results_df)] = ['Specificity (Test)']  
    PPAD_metrics_results_df.loc[len(PPAD_metrics_results_df)] = ['Accuracy (Train)']
    PPAD_metrics_results_df.loc[len(PPAD_metrics_results_df)] = ['Loss (Train)']
    PPAD_metrics_results_df.loc[len(PPAD_metrics_results_df)] = ['F-2 (Train)']
    
    return PPAD_metrics_results_df 

# Best hyperparameters that have been chosen by grid search optimization

In [48]:
# To read a csv file that contains best hyperparameters and copy it in a dataframe
# Hyperparameters df contains the best values of batch_size, epoch, dropout, l2, and RNN cell

file_name = '/kaggle/input/hp-df-csv-file/hp_df.csv'
PPAD_hp_df = read_csv(file_name,header=0)

In [49]:
PPAD_hp_df

,batch_size,epoch,dropout,l2,cell
0,8,50,0.4,0.001,GRU


# Global variables

In [50]:
# unpikle data

# Longitudinal training data
file_name = '/kaggle/input/time-diff-our-ppad-ae/longitudinal_data_train.pkl'
lon_data_train = pd.read_pickle(file_name)

# Labels of traing data 
file_name = '/kaggle/input/time-diff-our-ppad-ae/label_train.pkl'
label_train = pd.read_pickle(file_name)

# Demographic training data
file_name = '/kaggle/input/time-diff-our-ppad-ae/demographic_data_train.pkl'
dem_data_train = pd.read_pickle(file_name)

# Longitudinal test data
file_name = '/kaggle/input/time-diff-our-ppad-ae/longitudinal_data_test.pkl'
lon_data_test = pd.read_pickle(file_name)

# Labels of test data 
file_name = '/kaggle/input/time-diff-our-ppad-ae/label_test.pkl'
label_test = pd.read_pickle(file_name)

# Demographic test data
file_name = '/kaggle/input/our-ppad-ae/demographic_data_test.pkl'
dem_data_test = pd.read_pickle(file_name)

In [51]:
# This represents number of visits (time points) will be used in the training.
time_steps = lon_data_test[0].shape[1]

# This represents number of future visits ahead to predict 
future_time_s = label_test[0].shape[1]

# This represents how many featutes in each visit (longitudinal).
num_features_in_each_time_step = lon_data_test[0].shape[2]

# This represents how many demographic featutes (cross sectional).
demographic_features = len(dem_data_test[0][0])

In [52]:
time_steps = lon_data_test[0].shape[1]
time_steps

2

In [53]:
future_time_s = label_test[0].shape[1]
future_time_s

2

In [54]:
num_features_in_each_time_step

12

# Runing PPAD-AE 5 times for one scenario and save results

In [55]:
longitudinal_train_data = lon_data_train[0]
train_label = label_train[0]
longitudinal_test_data = lon_data_test[0]
test_label = label_test[0]
demographic_train_data = np.array(dem_data_train[0])
demographic_test_data = np.array(dem_data_test[0])

# HP
PPAD_hp_list = list(PPAD_hp_df.iloc[0,:])

PPAD_metrics_results_df = create_table()
for j in range(5):
    print("iteration_", j+1)
    #PPAD-AE
    PPAD_result = do_PPAD_AE(longitudinal_train_data, train_label, longitudinal_test_data, test_label, demographic_train_data,
                          demographic_test_data, j+1, future_time_s, PPAD_hp_list)
    PPAD_metrics_results_df = pd.concat([PPAD_metrics_results_df, PPAD_result], axis=1)
    print("PPAD-AE")
# SAVE RESULTS
PPAD_scenario = str(time_steps)+'_'+str(future_time_s)+'_PPAD-AE.csv'
PPAD_metrics_results_df.to_csv(PPAD_scenario, index = False)

iteration_ 1
LSTM
Training is over
Test is over
predicted_l
[0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1.
 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0.
 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1.
 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.]
real_l
[0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1.
 0. 1. 0. 0. 1. 0. 1. 